In [ ]:
# !pip install langchain langchain_community langchain_openai langchain_text_splitters chromadb python-dotenv gdown pypdf crewai crewai_tools pandas pydantic

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.2)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

# Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents # Stored in documents

extracted_data = load_pdf_file(data='./data/')

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # Select the chunking strategy

# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large",
                                   api_key= os.getenv('OPENAI_API_KEY'))

vectordb = Chroma.from_texts([t.page_content for t in text_chunks],
                             embeddings,
                             collection_name="bankpolicy",
                             persist_directory="./bankpolicy_db")
retriever = vectordb.as_retriever(search_kwargs={ "k" : 3})

In [ ]:
llm = ChatOpenAI(
    openai_api_base="https://api.openai.com/v1",
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    model_name="gpt-4",
    temperature=0.2,
    max_tokens=1000,
)

In [ ]:
import pandas as pd

# --- 1. Simulated Credit Score System ---
credit_data = {
    "ID": ["1111", "2222", "3333", "4444", "5555"],
    "Name": ["Loren", "Matt", "Hilda", "Andy", "Kit"],
    "Credit_Score": [455, 685, 825, 840, 350]
}
credit_df = pd.DataFrame(credit_data)

# --- 2. Simulated Account Status System ---
account_data = {
    "ID": ["1111", "2222", "3333", "4444", "5555"],
    "Name": ["Loren", "Matt", "Hilda", "Andy", "Kit"],
    "Nationality": ["Singaporean", "Non-Singaporean", "Singaporean", "Non-Singaporean", "Singaporean"],
    "Account_Status": ["good-standing", "closed", "delinquent", "good-standing", "delinquent"]
}
account_df = pd.DataFrame(account_data)

# --- 3. Simulated Government PR System ---
pr_data = {
    "ID": ["2222", "4444", "1111", "3333", "5555"],
    "Name": ["Matt", "Andy", "Loren", "Hilda", "Kit"],
    "PR_Status": ["true", "false", "true", "true", "false"]
}
pr_df = pd.DataFrame(pr_data)

print("✅ Mock Banking Systems (Databases) Initialized.")

In [ ]:
from crewai.tools import BaseTool
from pydantic import Field
from langchain_core.documents import Document

# Define Tools
class CustomerIDLookupTool(BaseTool):
    name: str = "Customer_ID_Lookup"
    description: str = (
        "Resolves a customer name to a customer ID using internal records. "
        "If multiple matches exist, returns all matching IDs. "
        "If no match exists, returns an explicit error message."
    )

    def _run(self, customer_name: str) -> str:
        # Case-insensitive exact match
        matches = credit_df[
            credit_df["Name"].str.lower() == customer_name.strip().lower()
        ]

        if matches.empty:
            return (
                f"ERROR: No customer found with name '{customer_name}'. "
                "Do not proceed with data retrieval."
            )

        if len(matches) > 1:
            ids: List[str] = matches["ID"].tolist()
            return (
                f"ERROR: Multiple customers found with name '{customer_name}'. "
                f"Matching Customer IDs: {ids}. "
                "Disambiguation required."
            )

        customer_id = matches.iloc[0]["ID"]
        return customer_id

class CreditScoreTool(BaseTool):
    name: str = "Get_Credit_Score"
    description: str = "Fetches credit score for a given Customer ID."

    def _run(self, customer_id: str) -> str:
        record = credit_df[credit_df["ID"] == customer_id]

        if record.empty:
            return f"No credit score found for Customer ID {customer_id}"

        score = int(record.iloc[0]["Credit_Score"])
        name = record.iloc[0]["Name"]

        return f"Customer: {name}, Credit Score: {score}"

class AccountStatusTool(BaseTool):
    name: str = "Get_Account_Status"
    description: str = "Fetches nationality and account status for a given Customer ID."

    def _run(self, customer_id: str) -> str:
        record = account_df[account_df["ID"] == customer_id]

        if record.empty:
            return f"No account record found for Customer ID {customer_id}"

        name = record.iloc[0]["Name"]
        nationality = record.iloc[0]["Nationality"]
        status = record.iloc[0]["Account_Status"]

        return (
            f"Customer: {name}, "
            f"Nationality: {nationality}, "
            f"Account Status: {status}"
        )
class PRStatusTool(BaseTool):
    name: str = "Get_PR_Status"
    description: str = "Fetches Permanent Residency (PR) status for a given Customer ID."

    def _run(self, customer_id: str) -> str:
        record = pr_df[pr_df["ID"] == customer_id]

        if record.empty:
            return f"No PR record found for Customer ID {customer_id}"

        name = record.iloc[0]["Name"]
        pr_status = record.iloc[0]["PR_Status"]

        return f"Customer: {name}, PR Status: {pr_status}"

class CustomerProfileTool(BaseTool):
    name: str = "Get_Customer_Profile"
    description: str = (
        "Fetches credit score, nationality, account status, and PR status "
        "for a given Customer ID."
    )

    def _run(self, customer_id: str) -> str:
        credit = credit_df[credit_df["ID"] == customer_id]
        account = account_df[account_df["ID"] == customer_id]
        pr = pr_df[pr_df["ID"] == customer_id]

        if credit.empty or account.empty:
            return f"Incomplete profile for Customer ID {customer_id}"

        name = credit.iloc[0]["Name"]
        credit_score = int(credit.iloc[0]["Credit_Score"])
        nationality = account.iloc[0]["Nationality"]
        account_status = account.iloc[0]["Account_Status"]
        pr_status = pr.iloc[0]["PR_Status"] if not pr.empty else "Not Available"

        return (
            f"Customer: {name}\n"
            f"Customer ID: {customer_id}\n"
            f"Credit Score: {credit_score}\n"
            f"Nationality: {nationality}\n"
            f"PR Status: {pr_status}\n"
            f"Account Status: {account_status}"
        )
class PolicySearchTool(BaseTool):
    name: str = "BankPolicy_Search"
    description: str = (
        "Retrieves relevant bank policy information (risk, eligibility, interest rates) "
        "from the prebuilt Chroma vector store."
    )

    retriever: any

    def _run(self, query: str) -> str:
        # Use the new LC API
        docs: List[Document] = self.retriever.invoke(query)
        if not docs:
            return "No relevant bank policy information found."

        return "\n\n".join(
            f"Policy Excerpt {i+1}:\n{doc.page_content}"
            for i, doc in enumerate(docs)
        )

customer_id_tool = CustomerIDLookupTool()
credit_score_tool = CreditScoreTool()
account_status_tool = AccountStatusTool()
pr_status_tool = PRStatusTool()
customer_profile_tool = CustomerProfileTool()
policy_search_tool = PolicySearchTool(retriever=retriever)

In [ ]:
from crewai import Agent, Task, Crew

# Define Agents
Data_Agent = Agent(
    role='Loan Applicant Data Retriever',
    goal='Retrieve loan applicant details based on customer name and customer ID',
    backstory=(
        'Use the available tools to retrieve the applicant’s credit score, account status, nationality, and PR status, as these details are essential for the next agent to determine the loan’s interest rate and risk level.\nSearches can be conducted using either the customer name or customer ID.\n\nAvailable systems:\n\nCredit Score System — Retrieve the applicant’s credit score.\n\nAccount Status System — Verify account status and nationality.\n\nGovernment PR Status System — Confirm Permanent Resident (PR) status for non-Singaporean applicants.\n\nEnsure all retrieved data is accurate and complete.\nReturn the output in JSON format with the following structure:\n\n{\n\"customer_name\": \"\",\n\"customer_id\": \"\",\n\"customer_email\": \"\",\n\"nationality\": \"\", // Singaporean or Non-Singaporean\n\"pr_status\": \"\", // True, False, or Not Applicable (for Non-Singaporean)\n\"credit_score\": \"\",\n\"account_status\": \"\" // e.g., good-standing, delinquent, closed\n}'
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

Policy_Agent = Agent(
    role='Bank Policy Retriever',
    goal='Retrieve loan applicant interest rates and risks based on the Data Retriever agent',
    backstory='Use information about the loan applicant, including credit score, account status, and PR status from the previous agent, to determine the appropriate loan interest rate and risk level based on the knowledge base.\n\nFollow the steps below:\n\nDetermine Overall Risk:\nUsing the applicant’s credit score and account status, reference the Bank Loan Overall Risk knowledge base to assess the applicant’s overall risk category.\n\nBased on the overall risk category, identify the applicable interest rate using the Bank Loan Overall Risk knowledge base.\n\nReturn the results in JSON format:\n\n{\n\"overall_risk\": \"\", // Determined from Bank Loan Overall Risk knowledge base\n\"loan_interest_rate\": \"\", // Determined from Bank Loan Rate knowledge base\n\"customer_name\": \"\",\n\"customer_email\": \"\",\n\"nationality\": \"\",\n\"pr_status\": \"\"\n}',
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

Report_Agent = Agent(
    role='Loan Recommendation Agent',
    goal='Generate loan recommendation based on the information from Policy_Agent and Data_Agent',
    backstory='After completing the assessment from the previous 2 Agents, generate a loan recommendation report that includes:\n\nOverall risk level\n\nApplicable interest rate\n\nPR status\n\nCustomer Name and Email\n\nRecommendations: Depending on the overall risk, interest rates, nationality and PR status (for non-Singaporean)\n\nOutput Format:\nCustomer Name:\nCustomer Email:\nOverall Risk:\nInterest Rate: %\nNationality: Singaporean or Non-Singaporean\nPR Status: True, False',
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [ ]:
# Define Tasks
data_retrieval_task = Task(
    description=(
        "Retrieve loan applicant data.\n\n"
        "Input:\n"
        "- Customer Name: {customer_name}\n"
        "- Customer ID: (optional)\n\n"
        "Rules:\n"
        "1. If Customer ID is not provided, resolve it using customer_id_tool.\n"
        "2. Do NOT guess Customer ID.\n"
        "3. If multiple IDs are found, stop and return an error message.\n"
        "4. Once Customer ID is resolved, retrieve credit score, nationality, "
        "account status, and PR status.\n\n"
        "Return output strictly in JSON format:\n\n"
        "{\n"
        "  \"customer_name\": \"\",\n"
        "  \"customer_id\": \"\",\n"
        "  \"customer_email\": \"\",\n"
        "  \"nationality\": \"\",\n"
        "  \"pr_status\": \"\",\n"
        "  \"credit_score\": \"\",\n"
        "  \"account_status\": \"\"\n"
        "}"
    ),
    expected_output="Valid JSON or explicit error message.",
    agent=Data_Agent,
    tools=[
        customer_id_tool,
        credit_score_tool,
        account_status_tool,
        pr_status_tool,
    ],
)
policy_assessment_task = Task(
    description=(
        "Using the customer profile JSON from the Data Retriever Agent:\n\n"
        "Steps:\n"
        "1. Determine the applicant’s overall risk level using credit score and account status.\n"
        "2. Identify the applicable loan interest rate based on the Bank Loan Policy knowledge base.\n"
        "3. Consider PR status for Non-Singaporean applicants.\n\n"
        "Return the output strictly in the following JSON format:\n\n"
        "{\n"
        "  \"overall_risk\": \"\",\n"
        "  \"loan_interest_rate\": \"\",\n"
        "  \"customer_name\": \"\",\n"
        "  \"customer_email\": \"\",\n"
        "  \"nationality\": \"\",\n"
        "  \"pr_status\": \"\"\n"
        "}"
    ),
    expected_output="Valid JSON with risk category and loan interest rate.",
    agent=Policy_Agent,
    context=[data_retrieval_task],
    tools=[policy_search_tool],
)

loan_recommendation_task = Task(
    description=(
        "Generate a loan recommendation report using:\n"
        "- Customer profile from Data Retriever Agent\n"
        "- Risk and interest rate assessment from Policy Agent\n\n"
        "Include:\n"
        "- Customer Name\n"
        "- Customer Email\n"
        "- Overall Risk Level\n"
        "- Interest Rate (%)\n"
        "- Nationality\n"
        "- PR Status\n"
        "- Clear loan recommendation based on risk and eligibility\n\n"
        "Use the following output format:\n\n"
        "Customer Name:\n"
        "Customer Email:\n"
        "Overall Risk:\n"
        "Interest Rate: %\n"
        "Nationality:\n"
        "PR Status:\n"
        "Recommendation:"
    ),
    expected_output="Well-structured loan recommendation report.",
    agent=Report_Agent,
    context=[data_retrieval_task, policy_assessment_task],
)


In [ ]:
# Define Crew
crew = Crew(
    agents=[Data_Agent, Policy_Agent, Report_Agent],
    tasks=[
        data_retrieval_task,
        policy_assessment_task,
        loan_recommendation_task,
    ],
    verbose=True,
)
# Run example
result = crew.kickoff(
    inputs={
        "customer_name": "Loren"
    }
)

print("Result:")
print(result)